# Shrinking HTML
idea: Maybe we can downalod the entire html page, cut it to the values on screen and that will fit the LLM context window?

In [42]:
all_elements_script = """
function getXPath(element) {
  if (element && element.nodeType === Node.ELEMENT_NODE) {
    const idx = Array.from(element.parentNode.children).indexOf(element) + 1;
    const tag = element.tagName.toLowerCase();
    const parentXPath = getXPath(element.parentNode);
    return `${parentXPath}/${tag}[${idx}]`;
  }
  return '';
}

// Get all elements in the HTML page
const elements = document.getElementsByTagName('*');

// Create an array to store the element details
const elementDetails = [];

// Iterate through each element
for (let i = 0; i < elements.length; i++) {
  const element = elements[i];

  // Get the bounding rectangle of the element
  const rect = element.getBoundingClientRect();

  // Get the text content of the element
  const textContent = element.textContent;

  // Get the tooltip value if it exists
  const tooltip = element.hasAttribute('title') ? element.getAttribute('title') : '';

  // Get the aria-label value
  const ariaLabel = (element.hasAttribute('aria-label') ? element.getAttribute('aria-label') : '');

  // Get the nodeName
  const e_type = element.nodeName;

  // Get the data-initial-value
  const data_initial_value = (element.hasAttribute('data-initial-value') ? element.getAttribute('data-initial-value') : '')

  // Get innerText
  const innerText = element.innerText

  // Store the element, its bounding rectangle, text content, and tooltip details
  const elementInfo = {
    element: element,
    rect: rect,
    textContent: textContent !== undefined ? textContent.trim().replaceAll(",",";"): "",
    ariaLabel: ariaLabel.replaceAll(",",";"),
    tooltip: tooltip.replaceAll(",",";"),
    e_type: e_type  !== undefined ? e_type.replaceAll(",",";"): "",
    data_initial_value: data_initial_value.replaceAll(",",";"),
    innerText: innerText !== undefined ? innerText.replaceAll(",",";"): "",
    parent_xpath : getXPath(element.parentElement)
  };

  const viewpointscroll = window.pageYOffset || document.documentElement.scrollTop;
  const viewportHeight = window.innerheight || document.documentElement.clientHeight;
  if (elementInfo.rect !== undefined && elementInfo.rect.height > 0 && elementInfo.rect.top >= viewpointscroll && elementInfo.rect.bottom < (viewpointscroll+viewportHeight)){
    elementDetails.push(elementInfo)
  }

}

// Display the element details
console.log("centerX,centerY,ElementType,textContent,TooltipValue,AriaLabel,data-initial-value,innerText,parent_xpath,height,width,top,bottom,left,right");
console.log(elementDetails.map( e=> (e.rect.left + (e.rect.width / 2))+","+(e.rect.top + (e.rect.height / 2))+","+e.e_type+","+e.textContent+","+e.tooltip+","+e.ariaLabel+","+e.data_initial_value+","+e.innerText+","+e.parent_xpath+","+e.rect.height+","+e.rect.width+","+e.rect.top+","+e.rect.bottom+","+e.rect.left+","+e.rect.right).join("\\n"));
"""


def run_js_code(wd,sub_script):
  script = f"""
  var consoleLogs = [];
  var originalLog = console.log;
  console.log = function(message) {{
      consoleLogs.push(message);
      originalLog.apply(console, arguments);
  }};

  {sub_script}

  return consoleLogs;
  """
  return wd.execute_script(script)

def screen_to_table(wd):
  import pandas as pd
  import io

  logs = run_js_code(wd,all_elements_script)
  try:
     return pd.read_csv(io.StringIO("\n".join(logs)), sep=",")
  except Exception as e:
    print(f"WARNING:\n On Table Data: {logs}\n Error {e}")
    return pd.read_csv(io.StringIO("\n".join(logs)), sep=",",on_bad_lines="skip")

In [43]:
def start_browesr():
  from selenium import webdriver
  from selenium.webdriver.chrome.service import Service

  
  chrome_options = webdriver.ChromeOptions()
  #chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--lang=en')
  #chrome_options.headless = True
  return webdriver.Remote("http://host.docker.internal:4444/wd/hub",options=chrome_options)
  #service = Service(executable_path=r'/usr/bin/chromedriver')
  #return webdriver.Chrome(service=service,options=chrome_options)

In [44]:
def web_driver_to_html(wd,file_name):
  # Get the page source HTML
  html_content = wd.page_source
  full_path = f"{file_name}.html"
  # Save the HTML content to a file
  with open(full_path, 'w', encoding='utf-8') as f:
      f.write(html_content)
  return full_path

In [45]:
try:
    web_driver = start_browesr()
    web_driver.get("https://www.google.com")
    web_driver_to_html(web_driver,"test")
    df = screen_to_table(web_driver)
finally:
    web_driver.quit()
df.head()

,centerX,centerY,ElementType,textContent,TooltipValue,AriaLabel,data-initial-value,innerText,parent_xpath,height,width,top,bottom,left,right
0,521,30,DIV,.Ne6nSd{display:flex;align-items:center;paddin...,NaN,NaN,NaN,Gmailחיפוש תמונות,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,כניסה,/html[1]/body[2]/div[2],60,1042,0,60,0.0,1042,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,521,30,DIV,Gmailחיפוש תמונותכניסה,NaN,NaN,NaN,Gmailחיפוש תמונות,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,כניסה,/html[1]/body[2]/div[2]/div[1],48,1030,6,54,6.0,1036,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,521,30,DIV,Gmailחיפוש תמונותכניסה,NaN,NaN,NaN,Gmailחיפוש תמונות,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
df.tail(1)

,centerX,centerY,ElementType,textContent,TooltipValue,AriaLabel,data-initial-value,innerText,parent_xpath,height,width,top,bottom,left,right
75,168.6015625,829,G-POPUP,הגדרות.EpPYLd{display:block;position:relative}...,NaN,NaN,NaN,הגדרות,/html[1]/body[2]/div[2]/div[5]/div[3]/div[3]/s...,16.0,73.484375,821.0,837.0,131.859375,205.34375


STOP: 
I've tried sending the www.google.com HTML file to the LLM, the Google site has no scroll options, so the entire HTML page is the HTML we will send the LLM if we 'Shrink it' according to the logic stated above.
We this simplest example, the token limit of gpt-3.5 was exceeded and this only a base site.

Conclusion: This technique will not work 